In [1]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import csv
import os
import time


In [2]:
# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sql_connect():
    engine = create_engine("mysql+pymysql://root:"
                +"big15"
                +"@localhost:3306/big15?charset=utf8"
                , encoding='utf8')
    
    return engine.connect()

In [5]:
# Data csv => SQL에 추가
def sql_data_insert(conn):
    
    # for문 돌려서 파일 리스트 받고 하나씩 읽는다
    for file in os.listdir('../data/update/20221116/concat/'):
        stock_code_name = file.split('_')[-2] + '_' + file.split('_')[-1].split('.')[0]

        stock_info = pd.read_csv('../data/update/20221116/concat/{0}'.format(file))

        stock_info['년'] = stock_info['날짜'].apply(str).str[:4]
        stock_info['월'] = stock_info['날짜'].apply(str).str[4:6]
        stock_info['일'] = stock_info['날짜'].apply(str).str[6:8]
        print(stock_info)
        # name= table명
        # con= connect할 때 들어가는 아이디 비밀번호 등등
        # if_exists= 'append' : 기존 테이블이 있는 경우 데이터를 추가
        # 'fail' : 기존 테이블이 있는 경우 아무일도 없지만 없을 경우 valuesError뜸
        # 'replace' : 기존 테이블이 있을 경우, 기존 테이블을 삭제하고 다시 테이블을
        count = stock_info.to_sql(name='{0}'.format(stock_code_name), con=conn, if_exists='replace', index=False)
        stock_info = pd.DataFrame()

        print(count)

    # for문 완료 후 연결 해제
    conn.close()


In [ ]:
sql_data_insert(sql_connect())

In [8]:
import pandas as pd

test = pd.read_csv('../data/update/20221116/concat/concat_000810_삼성화재.csv')


In [9]:
test['년'] = test['날짜'].apply(str).str[:4]
test['월'] = test['날짜'].apply(str).str[4:6]
test['일'] = test['날짜'].apply(str).str[6:8]
test

,날짜,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매도수량,전일대비,...,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,년,월,일
0,20221116,1530,193500.0,193500.0,193500.0,193500.0,15207.0,2.942550e+09,88718.0,-1500,...,23596984,23777853,50.189999,20221116,100.0,-35347,-916140,2022,11,16
1,20221116,1520,193500.0,193500.0,193000.0,193000.0,319.0,6.165000e+07,88718.0,-1500,...,23596984,23777853,50.189999,20221116,100.0,-35347,-916140,2022,11,16
2,20221116,1519,193000.0,193500.0,193000.0,193500.0,351.0,6.781000e+07,88570.0,-1500,...,23596984,23777853,50.189999,20221116,100.0,-35347,-916140,2022,11,16
3,20221116,1518,193000.0,193500.0,193000.0,193000.0,345.0,6.666000e+07,88345.0,-1500,...,23596984,23777853,50.189999,20221116,100.0,-35347,-916140,2022,11,16
4,20221116,1517,193000.0,193500.0,193000.0,193500.0,312.0,6.035000e+07,88165.0,-1500,...,23596984,23777853,50.189999,20221116,100.0,-35347,-916140,2022,11,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193804,20201028,905,189500.0,189500.0,189000.0,189000.0,608.0,1.149200e+08,1019.0,-1500,...,25318414,22056423,46.560001,20201028,100.0,5661,1014353,2020,10,28
193805,20201028,904,190000.0,190000.0,189500.0,189500.0,145.0,2.747000e+07,517.0,-1500,...,25318414,22056423,46.560001,20201028,100.0,5661,1014353,2020,10,28
193806,20201028,903,189500.0,189500.0,189000.0,189500.0,450.0,8.527000e+07,389.0,-1500,...,25318414,22056423,46.560001,20201028,100.0,5661,1014353,2020,10,28
193807,20201028,902,190000.0,190500.0,190000.0,190000.0,141.0,2.680000e+07,76.0,-1500,...,25318414,22056423,46.560001,20201028,100.0,5661,1014353,2020,10,28


In [13]:
import pymysql
import csv
import os
import time

conn = pymysql.connect(host='localhost', user='root', password='big15'
                     , db='big15', charset='utf8')

file_list = []
for file in os.listdir('../data/update/20221116/concat/'):
    curs = conn.cursor()
    file_list.append(file)
    stock_code_name = file.split('_')[-2] + '_' + file.split('_')[-1].split('.')[0]

    sql = 'CREATE TABLE {0} (date int)'.format(stock_code_name)
    curs.execute(sql)
    curs.close()

conn.commit()

# sql = "insert into user (id, name, region, insdt) values (%s, %s, %s, now())"
# f = open('test.csv', 'r', encoding='utf-8')
# rd = csv.reader(f)

# for line in rd:
#     curs.execute(sql, (line[0], line[1], line[2]))

# conn.commit()
# conn.close()
# f.close()







ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '-Oil (date int)' at line 1")

In [ ]:
import csv

f = open('../data/test1/000660_SK하이닉스.csv', 'r', encoding='utf-8')
rd = csv.reader(f)

for line in rd:
    print(line[1])

In [27]:
f.name.split('_')[1].split('.')[0]

'SK하이닉스'

In [10]:
host_name = "localhost"
username = 'root'
password = 'big15'
database_name = 'big15'

In [11]:
db1 = pymysql.connect(
    host= host_name
    , port=3306
    , user=username
    , password=password
    , db=database_name
    , charset='utf8'
)

In [12]:
sql = 'select * from stock'
df2 = pd.read_sql(sql,con=db1)
print(df2)

c:\Users\TJ\.conda\envs\tf2\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


        Unnamed: 0    시간     시가     고가     저가     종가  전일대비     거래량  \
0         20221103  1530  82700  82700  82700  82700     0  312503   
1         20221103  1520  83100  83100  83000  83100     0   12246   
2         20221103  1519  83000  83100  83000  83100     0   19656   
3         20221103  1518  83200  83200  83000  83000     0   39559   
4         20221103  1517  83100  83200  83100  83200     0    4789   
...            ...   ...    ...    ...    ...    ...   ...     ...   
190375    20201027   905  82000  82300  81800  82200     0   53979   
190376    20201027   904  82200  82300  82000  82000     0   19926   
190377    20201027   903  82200  82300  82100  82200     0   16356   
190378    20201027   902  82500  82500  82200  82200     0   12931   
190379    20201027   901  82300  82600  82200  82600     0   41258   

               거래대금  누적체결매도수량  ...  외국인현보유비율  수정주가일자  수정주가비율  기관순매수량  \
0       25843990000   1314512  ...       0.0       0     0.0       0   
1        101726

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190380 entries, 0 to 190379
Data columns (total 25 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  190380 non-null  int64  
 1   시간          190380 non-null  int64  
 2   시가          190380 non-null  int64  
 3   고가          190380 non-null  int64  
 4   저가          190380 non-null  int64  
 5   종가          190380 non-null  int64  
 6   전일대비        190380 non-null  int64  
 7   거래량         190380 non-null  int64  
 8   거래대금        190380 non-null  int64  
 9   누적체결매도수량    190380 non-null  int64  
 10  상장주식수       190380 non-null  int64  
 11  시가총액        190380 non-null  int64  
 12  외국인주문한도수량   190380 non-null  int64  
 13  외국인주문가능수량   190380 non-null  int64  
 14  외국인현보유수량    190380 non-null  int64  
 15  외국인현보유비율    190380 non-null  float64
 16  수정주가일자      190380 non-null  int64  
 17  수정주가비율      190380 non-null  float64
 18  기관순매수량      190380 non-null  int64  
 19  기관

In [16]:
df2 = df2.astype({'Unnamed: 0':'int32'})
df2 = df2.astype({'시간':'int16'})
df2 = df2.astype({'시가':'int32'})
df2 = df2.astype({'고가':'int32'})
df2 = df2.astype({'저가':'int32'})
df2 = df2.astype({'종가':'int32'})
df2 = df2.astype({'전일대비':'int32'})
df2 = df2.astype({'거래량':'int32'})
df2 = df2.astype({'거래대금':'int64'})
df2 = df2.astype({'누적체결매도수량':'int32'})
df2 = df2.astype({'상장주식수':'int32'})
df2 = df2.astype({'시가총액':'int64'})
df2 = df2.astype({'외국인주문한도수량':'int32'})
df2 = df2.astype({'외국인주문가능수량':'int32'})
df2 = df2.astype({'외국인현보유수량':'int32'})
df2 = df2.astype({'외국인현보유비율':'float32'})
df2 = df2.astype({'수정주가일자':'int32'})
df2 = df2.astype({'수정주가비율':'float32'})
df2 = df2.astype({'기관순매수량':'int32'})
df2 = df2.astype({'기관누적순매수량':'int32'})
df2 = df2.astype({'등락주선':'int32'})
df2 = df2.astype({'등락비율':'float32'})
df2 = df2.astype({'예탁금':'int64'})
df2 = df2.astype({'주식회전율':'float32'})
df2 = df2.astype({'거래성립률':'float32'})


In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190380 entries, 0 to 190379
Data columns (total 25 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  190380 non-null  int32  
 1   시간          190380 non-null  int16  
 2   시가          190380 non-null  int32  
 3   고가          190380 non-null  int32  
 4   저가          190380 non-null  int32  
 5   종가          190380 non-null  int32  
 6   전일대비        190380 non-null  int32  
 7   거래량         190380 non-null  int32  
 8   거래대금        190380 non-null  int64  
 9   누적체결매도수량    190380 non-null  int32  
 10  상장주식수       190380 non-null  int32  
 11  시가총액        190380 non-null  int64  
 12  외국인주문한도수량   190380 non-null  int32  
 13  외국인주문가능수량   190380 non-null  int32  
 14  외국인현보유수량    190380 non-null  int32  
 15  외국인현보유비율    190380 non-null  float32
 16  수정주가일자      190380 non-null  int32  
 17  수정주가비율      190380 non-null  float32
 18  기관순매수량      190380 non-null  int32  
 19  기관

In [18]:
sql = 'SELECT COUNT(*) FROM STOCK'
print(pd.read_sql(sql=sql, con=db1))

   COUNT(*)
0    190380


c:\Users\TJ\.conda\envs\tf2\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
